In [1]:
import geopandas as gpd
import pandas as pd
import os
import googlemaps
import datetime


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#load points
bikes = gpd.read_file('networkAnalysisResults/locationAllocation/bikesOutputLocationAllocation.shp')
bikes = bikes.to_crs(epsg=4326)
bikes = bikes.loc[:,['DemandCoun','DemandWeig','Name','ObjectID','TotalWeigh',
                    'Total_Leng','Weight','bikesID','geometry']]
bikes['x'] = bikes.geometry.map(lambda coord: coord.x)
bikes['y'] = bikes.geometry.map(lambda coord: coord.y)
bikes.head()



,DemandCoun,DemandWeig,Name,ObjectID,TotalWeigh,Total_Leng,Weight,bikesID,geometry,x,y
0,12,36.601939,1007,1008,7242.546427,3088.344297,1.0,1008,POINT (-58.48271903122684 -34.63372489684256),-58.482719,-34.633725
1,8,32.536800,1454,1455,6439.347252,1995.020737,1.0,1455,POINT (-58.47717057647716 -34.61421608467159),-58.477171,-34.614216
2,7,30.660645,1505,1506,3833.049079,1558.606123,1.0,1506,POINT (-58.47657764368105 -34.56641627994559),-58.476578,-34.566416
3,8,40.753438,1817,1818,8275.599561,1938.871795,1.0,1818,POINT (-58.47304090788498 -34.65855188127559),-58.473041,-34.658552
4,9,33.510633,1969,1970,5986.754399,2216.986577,1.0,1970,POINT (-58.4713938756251 -34.542134330909),-58.471394,-34.542134


In [3]:
homes = gpd.read_file('networkAnalysisResults/locationAllocation/bikeDemandHomes.shp')
homes = homes.to_crs(epsg=4326)
homes = homes.loc[:,['FacilityID','Name','ObjectID','Weight','geometry']]
homes['x'] = homes.geometry.map(lambda coord: coord.x)
homes['y'] = homes.geometry.map(lambda coord: coord.y)
homes.head()

,FacilityID,Name,ObjectID,Weight,geometry,x,y
0,1008,10_12_1,50,9.704001,POINT (-58.48433162373792 -34.63082960424205),-58.484332,-34.630830
1,1008,10_12_10,51,8.441567,POINT (-58.48394746240098 -34.63217952684114),-58.483947,-34.632180
2,1008,10_12_8,57,7.308287,POINT (-58.47998581546195 -34.632289801078),-58.479986,-34.632290
3,1008,10_12_9,58,7.614843,POINT (-58.48189895438345 -34.63262059571757),-58.481899,-34.632621
4,1008,10_13_11,60,8.401102,POINT (-58.48531894103925 -34.6322886077533),-58.485319,-34.632289


## Homes to bikes


In [4]:
homesToMerge = homes.loc[:,['Name','ObjectID','x','y']]
homesToMerge.columns=['homesName','DemandID','home_x','home_y']

bikesToMerge = bikes.loc[:,['Name','ObjectID','x','y']]
bikesToMerge.columns = ['bikesName','FacilityID','bike_x','bike_y']

In [5]:
homesToBikes = gpd.read_file('networkAnalysisResults/locationAllocation/linesHomeToBike.shp')
#Total_Leng es la cantidad en metros que recorre
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,Total_Leng,Weight,geometry
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...


In [6]:
homesToBikes = pd.merge(homesToBikes,homesToMerge, on=['DemandID'],how='left')
homesToBikes = pd.merge(homesToBikes,bikesToMerge, on=['FacilityID'],how='left')
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,Total_Leng,Weight,geometry,homesName,home_x,home_y,bikesName,bike_x,bike_y
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...,10_12_1,-58.484332,-34.630830,1007,-58.482719,-34.633725
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...,10_12_10,-58.483947,-34.632180,1007,-58.482719,-34.633725
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...,10_12_8,-58.479986,-34.632290,1007,-58.482719,-34.633725
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...,10_12_9,-58.481899,-34.632621,1007,-58.482719,-34.633725
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...,10_13_11,-58.485319,-34.632289,1007,-58.482719,-34.633725


In [7]:
key = os.getenv('GOOGLEAPI')
gmaps = googlemaps.Client(key=key)

In [8]:
googleDist = []
googleTime = []
for i in range(homesToBikes.shape[0]):
#for i in range(100):
    travel = gmaps.distance_matrix(origins = (homesToBikes.home_y[i],homesToBikes.home_x[i]),
                               destinations = (homesToBikes.bike_y[i],homesToBikes.bike_x[i]),
                               mode="walking")
    googleDist.append(travel['rows'][0]['elements'][0]['distance']['value'])
    googleTime.append(travel['rows'][0]['elements'][0]['duration']['value']*1./60)
   


In [9]:
homesToBikes['googleDist'] = googleDist
homesToBikes['googleTime'] = googleTime

In [10]:
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,Total_Leng,Weight,geometry,homesName,home_x,home_y,bikesName,bike_x,bike_y,googleDist,googleTime
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...,10_12_1,-58.484332,-34.630830,1007,-58.482719,-34.633725,375,4.566667
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...,10_12_10,-58.483947,-34.632180,1007,-58.482719,-34.633725,209,2.600000
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...,10_12_8,-58.479986,-34.632290,1007,-58.482719,-34.633725,399,4.883333
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...,10_12_9,-58.481899,-34.632621,1007,-58.482719,-34.633725,273,3.383333
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...,10_13_11,-58.485319,-34.632289,1007,-58.482719,-34.633725,320,4.066667


In [11]:
homesToBikes.columns = ['DemandID','FacilityID','Name','ObjectID','TotalWeigh','homTbikeDist',
 'Weight','geometry','homesName','home_x','home_y','bikesName','bike_x','bike_y',
 'googleDist','homTbikeTime']

In [12]:
homesToBikes.to_file('networkAnalysisResults/homeToDowntown/homeToDowntown_homeToBike.shp')

## Bikes to Subway

In [13]:
bikes.head()

,DemandCoun,DemandWeig,Name,ObjectID,TotalWeigh,Total_Leng,Weight,bikesID,geometry,x,y
0,12,36.601939,1007,1008,7242.546427,3088.344297,1.0,1008,POINT (-58.48271903122684 -34.63372489684256),-58.482719,-34.633725
1,8,32.536800,1454,1455,6439.347252,1995.020737,1.0,1455,POINT (-58.47717057647716 -34.61421608467159),-58.477171,-34.614216
2,7,30.660645,1505,1506,3833.049079,1558.606123,1.0,1506,POINT (-58.47657764368105 -34.56641627994559),-58.476578,-34.566416
3,8,40.753438,1817,1818,8275.599561,1938.871795,1.0,1818,POINT (-58.47304090788498 -34.65855188127559),-58.473041,-34.658552
4,9,33.510633,1969,1970,5986.754399,2216.986577,1.0,1970,POINT (-58.4713938756251 -34.542134330909),-58.471394,-34.542134


In [14]:
#homeToBikes.FacilityID es el id de las bikes 1008 etc
bikeToSubway = gpd.read_file('networkAnalysisResults/bikesToSubwayQGIS/routes.shp')
bikeToSubway = bikeToSubway.loc[:,['Name','Total_Leng']]

#separar name por -
#pasar el item 1 de esa lista a int
bikeToSubway['Name'] = bikeToSubway['Name'].map(lambda x: int((x.split()[2])))
bikeToSubway.columns=['FacilityID','bikeTsubDist']
#10km/h 10 is the velocity in kilometers
bikeToSubway['bikeTsubTime']= bikeToSubway.bikeTsubDist/1000/10*60 

bikeToSubway.head()

,FacilityID,bikeTsubDist,bikeTsubTime
0,1008,1336.253518,8.017521
1,1455,2104.245034,12.625470
2,1506,1487.526111,8.925157
3,1818,2313.674191,13.882045
4,1970,1879.183296,11.275100


In [15]:
#quedarme con Total_length y ese int
#llamarlo FacilityID, que es el id de las bikes
homesToBikes = pd.merge(homesToBikes,
                        bikeToSubway,
                        on=['FacilityID'],
                        how='left')


In [16]:
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,homTbikeDist,Weight,geometry,homesName,home_x,home_y,bikesName,bike_x,bike_y,googleDist,homTbikeTime,bikeTsubDist,bikeTsubTime
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...,10_12_1,-58.484332,-34.630830,1007,-58.482719,-34.633725,375,4.566667,1336.253518,8.017521
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...,10_12_10,-58.483947,-34.632180,1007,-58.482719,-34.633725,209,2.600000,1336.253518,8.017521
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...,10_12_8,-58.479986,-34.632290,1007,-58.482719,-34.633725,399,4.883333,1336.253518,8.017521
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...,10_12_9,-58.481899,-34.632621,1007,-58.482719,-34.633725,273,3.383333,1336.253518,8.017521
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...,10_13_11,-58.485319,-34.632289,1007,-58.482719,-34.633725,320,4.066667,1336.253518,8.017521


In [18]:
homesToBikes.to_file('networkAnalysisResults/homeToDowntown/homeToDowntown_homeToSubway.shp')

## From subway to downtown

In [97]:
bikeToSubwayRoute = gpd.read_file('networkAnalysisResults/bikesToSubwayQGIS/bikeToSubway_routes.shp')
bikeToSubwayRoute = bikeToSubwayRoute.loc[:,['FacilityID','Name']]
bikeToSubwayRoute.head()
#FacilityID is the id of the Facility of the subway

,FacilityID,Name
0,82,83 - 1008
1,82,83 - 1455
2,81,81 - 1506
3,67,67 - 1818
4,71,71 - 1970


In [98]:
subwayStations = gpd.read_file('networkAnalysisResults/bikesToSubwayQGIS/bikeToSubway_subwayFacilities.shp')
subwayStations = subwayStations.to_crs(epsg=4326)
#quiero los subtes que estan presentes en las rutas atadas a un bike
#bikeToSubwayRoute.FacilityID son las estaciones en la base de rutas de bike (facility) a subways

subwayStations = subwayStations.loc[(subwayStations.ObjectID.isin(bikeToSubwayRoute.FacilityID)),['ObjectID','geometry']] 
#tener las coordenadas de esas estaciones
subwayStations['x'] = subwayStations.geometry.map(lambda coord: coord.x)
subwayStations['y'] = subwayStations.geometry.map(lambda coord: coord.y)
subwayStations.drop(['geometry'],axis=1,inplace=True)
subwayStations.columns = ['FacilityID','subway_x','subway_y']
subwayStations.head()


,FacilityID,subway_x,subway_y
9,10,-58.415358,-34.585603
12,13,-58.373422,-34.591641
18,19,-58.380838,-34.628066
28,29,-58.439173,-34.599415
41,42,-58.435830,-34.618727


In [76]:
#get times from each station to downtown

downtown = (-34.605602,-58.374452)

lastMonday = datetime.datetime(2017, 5, 8, 8, 30)
depart_time = (lastMonday-datetime.datetime(1970,1,1)).total_seconds()

googleTimeSwubwayToDowntown = []
for i in range(subwayStations.shape[0]):

    travel = gmaps.distance_matrix(origins = (subwayStations.subway_y.iloc[i],subwayStations.subway_x.iloc[i]),
                                   destinations = downtown,
                                   mode="transit",
                                   transit_mode = 'subway',
                                   traffic_model="best_guess",
                                   departure_time = depart_time,
                                   transit_routing_preference = 'fewer_transfers')

    googleTimeSwubwayToDowntown.append(travel['rows'][0]['elements'][0]['duration']['value']*1./60)





In [99]:
subwayStations['subTdownTime'] = googleTimeSwubwayToDowntown
subwayStations.head()

,FacilityID,subway_x,subway_y,subTdownTime
9,10,-58.415358,-34.585603,15.666667
12,13,-58.373422,-34.591641,15.816667
18,19,-58.380838,-34.628066,13.433333
28,29,-58.439173,-34.599415,19.283333
41,42,-58.435830,-34.618727,22.133333


In [100]:
#vinculada cada estacion con una bike para mergear contra la base madre
bikeToSubwayRoute = pd.merge(bikeToSubwayRoute,subwayStations,
                             on = ['FacilityID'],
                             how='left')
bikeToSubwayRoute.head()

,FacilityID,Name,subway_x,subway_y,subTdownTime
0,82,83 - 1008,-58.469040,-34.631154,31.283333
1,82,83 - 1455,-58.469040,-34.631154,31.283333
2,81,81 - 1506,-58.485785,-34.574767,32.516667
3,67,67 - 1818,-58.461052,-34.643758,31.883333
4,71,71 - 1970,-58.461779,-34.556090,28.950000


In [101]:
#bikeToSubwayRoute['SubwayName'] = bikeToSubwayRoute['Name'].map(lambda x: int((x.split()[0])))
bikeToSubwayRoute['bikeName'] = bikeToSubwayRoute['Name'].map(lambda x: int((x.split()[2])))
bikeToSubwayRoute.drop(['FacilityID'],axis=1,inplace=True)


,FacilityID,Name,subway_x,subway_y,subTdownTime,bikeName
0,82,83 - 1008,-58.469040,-34.631154,31.283333,1008
1,82,83 - 1455,-58.469040,-34.631154,31.283333,1455
2,81,81 - 1506,-58.485785,-34.574767,32.516667,1506
3,67,67 - 1818,-58.461052,-34.643758,31.883333,1818
4,71,71 - 1970,-58.461779,-34.556090,28.950000,1970


In [105]:
bikeToSubwayRoute.columns = ['linkSubBike', 'subway_x', 'subway_y', 'subTdownTime', 'FacilityID']

In [106]:
bikeToSubwayRoute.head()

,linkSubBike,subway_x,subway_y,subTdownTime,FacilityID
0,83 - 1008,-58.469040,-34.631154,31.283333,1008
1,83 - 1455,-58.469040,-34.631154,31.283333,1455
2,81 - 1506,-58.485785,-34.574767,32.516667,1506
3,67 - 1818,-58.461052,-34.643758,31.883333,1818
4,71 - 1970,-58.461779,-34.556090,28.950000,1970


In [107]:
#en homesToBikes FacilityID es el ID de las bicis

homesToBikes = pd.merge(homesToBikes,bikeToSubwayRoute,
                        on=['FacilityID'],
                        how='left')

In [110]:
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,homTbikeDist,Weight,geometry,homesName,home_x,...,bike_x,bike_y,googleDist,homTbikeTime,bikeTsubDist,bikeTsubTime,linkSubBike,subway_x,subway_y,subTdownTime
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...,10_12_1,-58.484332,...,-58.482719,-34.633725,375,4.566667,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...,10_12_10,-58.483947,...,-58.482719,-34.633725,209,2.600000,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...,10_12_8,-58.479986,...,-58.482719,-34.633725,399,4.883333,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...,10_12_9,-58.481899,...,-58.482719,-34.633725,273,3.383333,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...,10_13_11,-58.485319,...,-58.482719,-34.633725,320,4.066667,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333


In [111]:
homesToBikes['totalTime'] = homesToBikes.homTbikeTime + homesToBikes.bikeTsubTime + homesToBikes.subTdownTime
homesToBikes.head()

,DemandID,FacilityID,Name,ObjectID,TotalWeigh,homTbikeDist,Weight,geometry,homesName,home_x,...,bike_y,googleDist,homTbikeTime,bikeTsubDist,bikeTsubTime,linkSubBike,subway_x,subway_y,subTdownTime,totalTime
0,50,1008,10_12_1 - 1007,462,403.220631,352.902583,1.142583,LINESTRING (98016.37760000024 99876.3146000001...,10_12_1,-58.484332,...,-34.633725,375,4.566667,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333,43.867521
1,51,1008,10_12_10 - 1007,463,841.126925,211.981898,3.967919,LINESTRING (98051.63680000044 99726.5665999995...,10_12_10,-58.483947,...,-34.633725,209,2.600000,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333,41.900854
2,57,1008,10_12_8 - 1007,464,365.552351,341.394647,1.070762,LINESTRING (98414.91650000028 99714.4025999996...,10_12_8,-58.479986,...,-34.633725,399,4.883333,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333,44.184188
3,58,1008,10_12_9 - 1007,465,760.523463,207.104278,3.672176,LINESTRING (98239.49149999954 99677.6737000001...,10_12_9,-58.481899,...,-34.633725,273,3.383333,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333,42.684188
4,60,1008,10_13_11 - 1007,466,642.480261,297.003681,2.163206,LINESTRING (97925.87679999974 99714.4381999997...,10_13_11,-58.485319,...,-34.633725,320,4.066667,1336.253518,8.017521,83 - 1008,-58.46904,-34.631154,31.283333,43.367521


In [112]:
homesToBikes.to_file('networkAnalysisResults/homeToDowntown/homeToDowntown_homeToDowntown.shp')

In [113]:
censusRadios = gpd.read_file('times/tiemposFinal2.shp')
censusRadios.head()

,CO_FRAC_RA,geometry,plata,secStd,tiemposMin,tiemposSec,x,y
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",7.50,1.199020,21.533333,1292,-58.368253,-34.587126
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",7.50,0.601368,10.800000,648,-58.385199,-34.605043
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",0.00,0.171687,3.083333,185,-58.376849,-34.605250
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",0.00,0.184679,3.316667,199,-58.371612,-34.605709
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",6.25,0.636633,11.433333,686,-58.383781,-34.604953


In [115]:
homesToBikesToMerge = homesToBikes.loc[:,['homesName','totalTime']]
homesToBikesToMerge.columns = ['CO_FRAC_RA','newTimeMin']
homesToBikesToMerge.head()

,CO_FRAC_RA,newTimeMin
0,10_12_1,43.867521
1,10_12_10,41.900854
2,10_12_8,44.184188
3,10_12_9,42.684188
4,10_13_11,43.367521


In [120]:
censusRadios = pd.merge(censusRadios,homesToBikesToMerge,
                        on=['CO_FRAC_RA'],
                        how='left')
censusRadios.head()

,CO_FRAC_RA,geometry,plata,secStd,tiemposMin,tiemposSec,x,y,newTimeMin
0,1_1_1,"POLYGON ((108330.8437499999 104181.0937500013,...",7.50,1.199020,21.533333,1292,-58.368253,-34.587126,NaN
1,1_12_1,"POLYGON ((107042.4609375 102801.421875, 107058...",7.50,0.601368,10.800000,648,-58.385199,-34.605043,NaN
2,1_12_10,"POLYGON ((107697.2187500009 102707.15625, 1078...",0.00,0.171687,3.083333,185,-58.376849,-34.605250,NaN
3,1_12_11,"POLYGON ((108748.546875 102668.9453125009, 108...",0.00,0.184679,3.316667,199,-58.371612,-34.605709,NaN
4,1_12_2,"POLYGON ((107169.9375 102810.3671875, 107300.3...",6.25,0.636633,11.433333,686,-58.383781,-34.604953,NaN


In [144]:
nonull = ~(censusRadios.newTimeMin.isnull())
justImprovement = censusRadios.copy()[nonull]


In [145]:
justImprovement.head()

,CO_FRAC_RA,geometry,plata,secStd,tiemposMin,tiemposSec,x,y,newTimeMin
307,10_12_1,"POLYGON ((97831.71875 99884.390625, 97902.3203...",11.5,2.31452,41.566667,2494,-58.484332,-34.630830,43.867521
308,10_12_10,"POLYGON ((97947.8359375 99787.96875, 98061.132...",11.5,2.21522,39.783333,2387,-58.483947,-34.632180,41.900854
315,10_12_8,"POLYGON ((98489.9296875 99777.8984375, 98505.8...",7.5,2.33401,41.916667,2515,-58.479986,-34.632290,44.184188
316,10_12_9,"POLYGON ((98217.671875 99745.6796875, 98331.24...",7.5,2.50477,44.983333,2699,-58.481899,-34.632621,42.684188
319,10_13_11,"POLYGON ((97915.9453125 99803.3125, 97957.6015...",11.5,2.29875,41.283333,2477,-58.485319,-34.632289,43.367521


In [146]:
justImprovement['diff'] = justImprovement.newTimeMin - justImprovement.tiemposMin 
justImprovement['diff'].describe()

count    193.000000
mean      -5.777158
std        5.809100
min      -25.967955
25%       -8.616884
50%       -4.794256
75%       -1.870963
max        6.794773
Name: diff, dtype: float64

In [149]:
justImprovement['diffP'] = justImprovement['diff']/justImprovement.tiemposMin*100
justImprovement['diffP'].describe()

count    193.000000
mean     -12.504060
std       12.539639
min      -43.408093
25%      -20.860888
50%      -11.212281
75%       -4.750757
max       27.155516
Name: diffP, dtype: float64

In [150]:
censusRadios.to_file('times/newTimes.shp')
justImprovement.to_file('times/newTimesImprov.shp')